In [ ]:
import boto3
import json

iam = boto3.client('iam', region_name='us-west-2')

In [ ]:
role_name = 'BedrockFlowsRole'
trust_policy = {
    "Version": "2012-10-17",
    "Statement": [{
        "Effect": "Allow",
        "Principal": {"Service": "bedrock.amazonaws.com"},
        "Action": "sts:AssumeRole"
    }]
}

bedrock_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
        "Sid": "BedrockFlowPermissions",
        "Effect": "Allow",
        "Action": [
            "bedrock:CreateFlow",
            "bedrock:UpdateFlow",
            "bedrock:GetFlow",
            "bedrock:ListFlows", 
            "bedrock:DeleteFlow",
            "bedrock:ValidateFlowDefinition", 
            "bedrock:CreateFlowVersion",
            "bedrock:GetFlowVersion",
            "bedrock:ListFlowVersions",
            "bedrock:DeleteFlowVersion",
            "bedrock:CreateFlowAlias",
            "bedrock:UpdateFlowAlias",
            "bedrock:GetFlowAlias",
            "bedrock:ListFlowAliases",
            "bedrock:DeleteFlowAlias",
            "bedrock:InvokeFlow",
            "bedrock:TagResource",
            "bedrock:UntagResource", 
            "bedrock:ListTagsForResource"
        ],
        "Resource": "*"
        },
        {
        "Sid": "BedrockResourcePermissions",
        "Effect": "Allow",
        "Action": [
            "bedrock:ApplyGuardrail",
            "bedrock:InvokeGuardrail",
            "bedrock:InvokeModel",
            "bedrock:GetCustomModel",
            "bedrock:InvokeAgent",
            "bedrock:Retrieve",
            "bedrock:RetrieveAndGenerate",
            "bedrock:GetPrompt",
            "bedrock:ListPrompts",
            "bedrock:RenderPrompt"
        ],
        "Resource": "*"
        },
        {
            "Sid": "GetBedrockResources",
            "Effect": "Allow",
            "Action": [
                "bedrock:GetAgent",
                "bedrock:GetKnowledgeBase",
                "bedrock:GetGuardrail",
                "bedrock:GetPrompt",
            ],
            "Resource": "*"
        }
    ]
}

try:
    # Try to get existing role
    role_response = iam.get_role(RoleName=role_name)
    role_arn = role_response['Role']['Arn']
    print(f"Using existing role: {role_arn}")
    
except iam.exceptions.NoSuchEntityException:
    # Role doesn't exist, create it
    print(f"Creating new role: {role_name}")
    role_response = iam.create_role(
        RoleName=role_name,
        AssumeRolePolicyDocument=json.dumps(trust_policy)
    )
    role_arn = role_response['Role']['Arn']
    
    # Attach Bedrock policy
    policy_name = 'BedrockFlowsPolicy'
    iam.put_role_policy(
        RoleName=role_name,
        PolicyName=policy_name,
        PolicyDocument=json.dumps(bedrock_policy)
    )
    
    print(f"Created role: {role_arn}")